# Chapter 4. Iterators and Generators

Iteration is one of Python’s strongest features.  
At a high level, you might simply view iteration as a way to process items in a sequence.  
However, there is so much more that is possible, such as creating your own iterator objects, applying useful iteration patterns in the itertools module, making generator functions, and so forth.  
This chapter aims to address common problems involving iteration.

## [Manually Consuming an Iterator](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_manually_consuming_an_iterator)

### Problem

You need to process items in an iterable, but for whatever reason, you can't or don't want to use a for loop.

### Solution

To manually consume an iterable, use the `next()` function and write your code to catch the `StopIteration` exception.  
This example manually reads lines from a file:

In [44]:
with open('python_ipsum.txt') as f:
    try:
        while True:
            line = next(f)
            print(line, end='')
    except StopIteration:
        pass
f.close()

Python Ipsum: Your source for Python-flavored placeholder text.
http://pythonipsum.com/

Lambda raspberrypi beautiful test script. Kwargs integration itertools dict reduce egg import cython.

Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python integration community. Object raspberrypi community bdfl cython import method.

Method raspberrypi diversity 2to3 return yield unit yield guido. Method integration mercurial unit import python exception dictionary. Django raspberrypi functools self import. Python integration mercurial dict return klass. Lambda integration mercurial 2to3 cython zen.

Import raspberrypi community pypi reduce dunder pyladies functools. Lambda raspberrypi decorator bd

Normally, `StopIteration` is used to signal the end of iteration.  
However, if you're using `next()` manually (as shown), you can also instruct it to return a terminating value, such as `None`, instead.

In [45]:
with open('python_ipsum.txt') as f:
    while True:
        line = next(f, None)
        if line is None:
            break
        print(line, end='')
f.close()

Python Ipsum: Your source for Python-flavored placeholder text.
http://pythonipsum.com/

Lambda raspberrypi beautiful test script. Kwargs integration itertools dict reduce egg import cython.

Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python integration community. Object raspberrypi community bdfl cython import method.

Method raspberrypi diversity 2to3 return yield unit yield guido. Method integration mercurial unit import python exception dictionary. Django raspberrypi functools self import. Python integration mercurial dict return klass. Lambda integration mercurial 2to3 cython zen.

Import raspberrypi community pypi reduce dunder pyladies functools. Lambda raspberrypi decorator bd

### Discussion

In most cases, the `for` statement is used to consume an iterable.  
However, every now and then, a problem calls for more precise control over the underlying iteration mechanism.  
Thus, it is useful to know what actually happens.  
The following example illustrates the basic mechanics of what happens during iteration:

In [46]:
items = [1, 2, 3]
# Get the iterator:
it = iter(items)  # Invokes items.__iter__()
# Run the iterator:
next(it)          # Invokes it.__next__()

1

In [47]:
next(it)

2

In [48]:
next(it)

3

Subsequent recipes in this chapter expand on iteration techniques, and knowledge of the basic iterator protocol is assumed.  
Be sure to tuck this first recipe away in your memory.

## [Delegating Iteration](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#delegate_iteration)

### Problem

You have built a custom container object that internally holds a list, tuple, or some other iterable.  
You would like to make iteration work with your new container.

### Solution

Typically, all you need to do is define an `__iter__()` method that delegates iteration to the internally held container.

In [49]:
class Node:
    
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)

Now for a demonstration:

In [50]:
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    for ch in root:
        print(ch)

Node(1)
Node(2)


In this code, the `__iter__()` method simply forwards the iteration request to the internally held `_children` attribute.

### Discussion

Python’s iterator protocol requires `__iter__()` to return a special iterator object that implements a `__next__()` method to carry out the actual iteration.  
If all you are doing is iterating over the contents of another container, you don’t really need to worry about the underlying details of how it works.  
All you need to do is to forward the iteration request along.  
The use of the `iter()` function here is a bit of a shortcut that cleans up the code.  
`iter(s)` simply returns the underlying iterator by calling `s.__iter__()`, much in the same way that `len(s)` invokes `s.__len__()`.

## [Creating New Iteration Patterns with Generators](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_problem_59)

### Problem

You want to implement a custom iteration pattern that’s different than the usual built-in functions (e.g., `range()`, `reversed()`, etc.).

### Solution

If you want to implement a new kind of iteration pattern, define it using a generator function.  
Here’s a generator that produces a range of floating-point numbers:

In [51]:
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x
        x += increment

To use such a function, you iterate over it using a for loop or use it with some other function that consumes an iterable (e.g., `sum()`, `list()`, etc.).

In [52]:
for n in frange(0, 4, 0.5):
    print(n)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5


In [53]:
# I want to try this with a list comprehension:
comp = [n for n in frange(0, 4, 0.5)]
print(comp)

[0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]


In [54]:
list(frange(0, 1, 0.125))

[0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875]

### Discussion

The mere presence of the yield statement in a function turns it into a generator.  
Unlike a normal function, a generator only runs in response to iteration.  
Here’s an experiment you can try to see the underlying mechanics of how such a function works:

In [55]:
def countdown(n):
    print('Starting to count from', n)
    while n > 0:
        yield n
        n -= 1
    print('Done, sucka!')

Create the generator and notice that no output appears:

In [56]:
c = countdown(3)
c

<generator object countdown at 0x102852a40>

Run the program to the first `yield` and emit a value.

In [57]:
next(c)

Starting to count from 3


3

Run the program to the next `yield` and repeat:

In [58]:
next(c)

2

In [59]:
next(c)

1

The key feature is that a generator function only runs in response to "next" operations carried out in iteration.  
Once a generator function returns, iteration stops.  
However, the `for` statement that’s usually used to iterate takes care of these details, so you don’t normally need to worry about them.

## [Implementing the Iterator Protocol](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_implementing_the_iterator_protocol)

### Problem

You are building custom objects on which you would like to support iteration, but would like an easy way to implement the iterator protocol.

### Solution

By far, the easieast way to implement iteration on an object is to use a generator function.  
In "Delegating Iteration", a `Node` class was presented for representing tree structures.  
Perhaps you want to implement an iterator that traverses nodes in a depth-first pattern.

In [60]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
    # https://www.pythoncentral.io/what-is-the-difference-between-__str__-and-__repr__-in-python/
    # https://docs.python.org/3.6/reference/datamodel.html#object.__repr__
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    # https://docs.python.org/3.6/library/stdtypes.html?highlight=__iter__#iterator.__iter__    
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()
            
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))
    
    for ch in root.depth_first():
        print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


In this code, the `depth_first()` method is fairly straightforward.  
It first yields itself and then iterates over each child yielding the items produced by the child's `depth_first()` method (using `yield from`).

### Discussion

Python’s iterator protocol requires `__iter__()` to return a special iterator object that implements a `__next__()` operation and uses a `StopIteration` exception to signal completion.  
However, implementing such objects can often be a messy affair.  
For example, the following code shows an alternative implementation of the `depth_first()` method using an associated iterator class:

In [61]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, other_node):
        self._children.append(other_node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        return DepthFirstIterator(self)
    
    
class DepthFirstIterator(object):
    """
    Depth-first traversal
    """
    def __init__(self, start_node):
        self._node = start_node
        self._children_iter = None
        self._child_iter = None
        
    def __iter__(self):
        return self
    
    def __next__(self):
        # Create an iterator for the children.
        if self._children_iter is None:
            self._children_iter = iter(self._node)
            return self._node
        
        # If processing a child, return its next item.
        elif self._child_iter:
            try:
                nextchild = next(self._child_iter)
                return nextchild
            except StopIteration:
                self._child_iter = None
                return next(self)
        # Advance to the next child and begin iteration.
        else:
            self._child_iter = next(self._children_iter).depth_first()
            return next(self)
        
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))

    for ch in root.depth_first():
        print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


The `DepthFirstIterator` class works in the same way as the generator version, but it’s a mess because the iterator has to maintain a lot of complex states about where it is in the iteration process.  
Frankly, nobody likes to write mind-bending code like that.  
Define your iterator as a generator and be done with it.

## [Iterating in Reverse](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_iterating_in_reverse)

### Problem

You want to iterate in reverse over a sequence.

### Solution

Use the built-in `reversed()` function.

In [62]:
a = [1, 2, 3, 4]
for x in reversed(a):
    print(x)

4
3
2
1


Reversed iteration only works if the object in question has a size that can be determined or if the object implements a `__reversed__()` special method.  
If neither of these can be satisfied, you’ll have to convert the object into a list first.

In [63]:
# Print a file backwards:
f = open('python_ipsum.txt')
for line in reversed(list(f)):
    print(line, end='')
f.close()

Method integration coroutine unit import cython implicit django python. Return raspberrypi functools pypi method gil dunder. Dunder raspberrypi decorator list reduce numpy. Future raspberrypi coroutine bdfl python egg gevent python functools descriptor.

Import raspberrypi community pypi reduce dunder pyladies functools. Lambda raspberrypi decorator bdfl. Python integration generator pypy script. Gevent integration generator pypy lambda jinja decorator arduino dict zip.

Method raspberrypi diversity 2to3 return yield unit yield guido. Method integration mercurial unit import python exception dictionary. Django raspberrypi functools self import. Python integration mercurial dict return klass. Lambda integration mercurial 2to3 cython zen.

Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs ra

Be aware that turning an iterable into a list as shown could consume a lot of memory if that iterable is large.

### Discussion

Many programmers don’t realize that reversed iteration can be customized on user-defined classes if they implement the `__reversed__()` method.

In [64]:
class Countdown:
    def __init__(self, start):
        self.start = start
        
    # Forward iterator:
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1
            
    # Reverse iterator:
    def __reversed__self():
        n = 1
        while n <= self.start:
            yield n
            n += 1

Defining a reversed iterator makes the code much more efficient, as it’s no longer necessary to pull the data into a list and iterate in reverse on the list.

## [Defining Generator Functions with Extra State](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_defining_generator_functions_with_extra_state)

### Problem

You would like to define a generator function, but it involves extra state that you would like to expose to the user somehow.

### Solution

If you want a genereator to expose extra state to the user, don't forget that you can easily implement it as a class, putting the generator function code in the `__iter__()` method:

In [65]:
from collections import deque

class Linehistory:
    
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)
        
    def __iter__(self):
        for line_number, line in enumerate(self.lines, 1):
            self.history.append((line_number, line))
            yield line

    def clear(self):
        self.history.clear()

To use this class, you would treat it like a normal generator function.  
However, since it creates an instance, you can access internal attributes, such as the `history` attribute or the `clear()` method.

In [66]:
with open('python_ipsum.txt') as f:
    lines = Linehistory(f)
    for line in lines:
        if 'python' in line:
            for line_number, hline in lines.history:
                print('{}:{}'.format(line_number, hline), end='')
f.close()

1:Python Ipsum: Your source for Python-flavored placeholder text.
2:http://pythonipsum.com/
4:Lambda raspberrypi beautiful test script. Kwargs integration itertools dict reduce egg import cython.
5:
6:Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python integration community. Object raspberrypi community bdfl cython import method.
6:Django integration functools unit object kwargs functools dictionary cython. Cython integration exception. Lambda integration diversity bdfl. Return integration exception self dunder. Python integration mercurial bdfl python lambda generator. Kwargs raspberrypi decorator unit cython import. Cython raspberrypi exception unit future klass exception. Python inte

### Discussion 

With generators, it is easy to fall into a trap of trying to do everything with functions alone.  
This can lead to rather complicated code if the generator function needs to interact with other parts of your program in unusual ways (exposing attributes, allowing control via method calls, etc.).  
If this is the case, just use a class definition, as shown.  
Defining your generator in the `__iter__()` method doesn’t change anything about how you write your algorithm.  
The fact that it’s part of a class makes it easy for you to provide attributes and methods for users to interact with. 

One potential subtlety with the method shown is that it might require an extra step of calling `iter()` if you are going to drive iteration using a technique other than a for loop.

In [67]:
f = open('python_ipsum.txt')
lines = Linehistory(f)
it = iter(lines)
print(next(it))
print(next(it))
print(next(it))
f.close()

Python Ipsum: Your source for Python-flavored placeholder text.

http://pythonipsum.com/





## [Taking a Slice of an Iterator](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#takingasliceofaniterator)

### Problem

You want to take a slice of data produced by an iterator, bu the normal slicing operator doesn't work.

### Solution

The `itertools.islice()` function is perfectly suited for taking slices of iterators and generators. 

Now let's try this using `islice()`:

In [68]:
import itertools

def count(n):
    while True:
        yield n
        n += 1

c = count(0)

for x in itertools.islice(c, 10, 20):
    print(x)

10
11
12
13
14
15
16
17
18
19


### Discussion

Iterators and generators can’t normally be sliced, because no information is known about their length (and they don’t implement indexing).  
The result of `islice()` is an iterator that produces the desired slice items, but it does this by consuming and discarding all of the items up to the starting slice index.  
Further items are then produced by the `islice` object until the ending index has been reached.

It’s important to emphasize that `islice()` will consume data on the supplied iterator.  
Since iterators can’t be rewound, that is something to consider.  
If it’s important to go back, you should probably just turn the data into a list first.

## [Skipping the First Part of an Iterable](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_skipping_the_first_part_of_an_iterable)

### Problem

You want to iterate over items in an iterable, but the first few items aren't of interest and you just want to discard them.

### Solution

The `itertools` module has a few functions that can be used to address this task.  
The first is the `itertools.dropwhile()` function.  
To use it, you supply a function and an iterable.  
The returned iterator discards the first items in the sequence as long as the supplied function returns `True`.  
Afterward, the entirety of the sequence is produced.  
To illustrate, suppose you are reading a file that starts with a series of comment lines:

If you want to skip all of the initial comment lines, here's one way to do it:

This example is based on skipping the first items according to a test function.  
If you happen to know the exact number of items that you want to skip, then you can use `itertools.islice()` instead.

In [69]:
from itertools import islice

items = ['a', 'b', 'c', 1, 4, 10, 15]
for x in islice(items, 3, None):
    print(x)

1
4
10
15


In this example, the last `None` argument to `islice()` is required to indicate that you want everything beyond the first three items as opposed to only the first three items (e.g., a slice of [3:] as opposed to a slice of [:3]).

### Discussion

The `dropwhile()` and `islice()` functions are mainly convenience functions that you can use to avoid writing rather messy code such as this:

Discarding the first part of an iterable is also slightly different than filtering all of it.  
For example, the first part of this recipe might be rewritten as follows:

This will obviously discard the comment lines at the start, but will also discard all such lines throughout the entire file.  
On the other hand, the solution only discards items until an item no longer satisfies the supplied test.  
After that, all subsequent items are returned with no filtering.  
Last, but not least, it should be emphasized that this recipe works with all iterables, including those whose size can’t be determined in advance.  
This includes generators, files, and similar kinds of objects.

## [Iterating Over All Possible Combinations or Permutations](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_iterating_over_all_possible_combinations_or_permutations)

### Problem

You want to iterate over all of the possible combinations or permutations of a collection of items.

### Solution

The `itertools` module provides three functions for this task.  
The first of these — `itertools.permutations()` — takes a collection of items and produces a sequence of tuples that rearranges all of the items into all possible permutations (i.e., it shuffles them into all possible configurations).

In [70]:
from itertools import permutations

items = ['a', 'b', 'c']
for p in permutations(items):
    print(p)

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')


If you want all permutations of a smaller length, you can give an optional length argument:

In [71]:
for p in permutations(items, 2):
    print(p)

('a', 'b')
('a', 'c')
('b', 'a')
('b', 'c')
('c', 'a')
('c', 'b')


You can also use `itertools.combinations()` to produce a sequence of combinations of items taken from the input:

In [72]:
from itertools import combinations

for c in combinations(items, 3):
    print(c)

('a', 'b', 'c')


In [73]:
for c in combinations(items, 2):
    print(c)

('a', 'b')
('a', 'c')
('b', 'c')


In [74]:
for c in combinations(items, 1):
    print(c)

('a',)
('b',)
('c',)


For `combinations()`, the actual order of the elements is not considered.  
That is, the combination ('a', 'b') is considered to be the same as ('b', 'a') (which is not produced).  
When producing combinations, chosen items are removed from the collection of possible candidates (i.e., if 'a' has already been chosen, then it is removed from consideration).  
The `itertools.combinations_with_replacement()` function relaxes this, and allows the same item to be chosen more than once.

In [75]:
from itertools import combinations_with_replacement

for c in combinations_with_replacement(items, 3):
    print(c)

('a', 'a', 'a')
('a', 'a', 'b')
('a', 'a', 'c')
('a', 'b', 'b')
('a', 'b', 'c')
('a', 'c', 'c')
('b', 'b', 'b')
('b', 'b', 'c')
('b', 'c', 'c')
('c', 'c', 'c')


### Discussion

This recipe demonstrates only some of the power found in the `itertools` module.  
Although you could certainly write code to produce permutations and combinations yourself, doing so would probably require more than a fair bit of thought.  
When faced with seemingly complicated iteration problems, it always pays to look at `itertools` first.  
If the problem is common, chances are a solution is already available.

## [Iterating Over the Index-Value Pairs of a Sequence](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_iterating_over_the_index_value_pairs_of_a_sequence)

### Problem

You want to iterate over a sequence while keeping track of which element of the sequence is currently being processed.

### Solution

The built-in `enumerate()` function handles this quite nicely:

In [76]:
my_list = ['a', 'b', 'c']
for idx, val in enumerate(my_list):
    print(idx, val)

0 a
1 b
2 c


If you want to start counting with a number other than 0, you can pass in a `start` argument:

In [77]:
another_list = ['d', 'e', 'f']
for idx, val in enumerate(another_list, 1):
    print(idx, val)

1 d
2 e
3 f


This case is especially useful for tracking line numbers in files if you want to use a line number in an error message:

In [78]:
def parse_data(filename):
    with open(filename, 'rt') as f:
        for line_number, line in enumerate(f, 1):
            fields = line.split()
            try:
                count = int(fields[1])
                # ...
                # ...
                pass
            except ValueError as e:
                print('Line {}: Parse error -- {}'.format(line_number, e))
    f.close()

The `enumerate()` function can be handy for keeping track of the offset into a list for occurrences of certain values, for example.  
So, if you want to map words in a file to the lines in which they occur, it can easily be accomplished using `enumerate()` to map each word to the line offset in the file where it was found:  

In [79]:
from collections import defaultdict

word_summary = defaultdict(list)

with open('python_ipsum.txt', 'r') as f:
    lines = f.readlines()
f.close()

for idx, line in enumerate(lines):
    # Create a list of words in the current line:
    words = [w.strip().lower() for w in line.split()]
    for word in words:
        word_summary[word].append(idx)

If you print `word_summary` after processing the file, it will be a dictionary (a defaultdict to be precise) that has a key for each word.  
The value for each word-key will be a list of line numbers that word occurred on.  
If the word occurred twice on a single line, that line number will be listed twice, making it possible to identify various simple metrics about the text.

In [80]:
print(word_summary)

defaultdict(<class 'list'>, {'python': [0, 5, 5, 5, 7, 7, 9, 11, 11], 'ipsum:': [0], 'your': [0], 'source': [0], 'for': [0], 'python-flavored': [0], 'placeholder': [0], 'text.': [0], 'http://pythonipsum.com/': [1], 'lambda': [3, 5, 5, 7, 9, 9], 'raspberrypi': [3, 5, 5, 5, 7, 7, 9, 9, 11, 11, 11], 'beautiful': [3], 'test': [3], 'script.': [3, 9], 'kwargs': [3, 5, 5], 'integration': [3, 5, 5, 5, 5, 5, 5, 7, 7, 7, 9, 9, 11], 'itertools': [3], 'dict': [3, 7, 9], 'reduce': [3, 9, 11], 'egg': [3, 11], 'import': [3, 5, 7, 9, 11], 'cython.': [3, 5], 'django': [5, 7, 11], 'functools': [5, 5, 7, 11, 11], 'unit': [5, 5, 5, 7, 7, 11], 'object': [5, 5], 'dictionary': [5], 'cython': [5, 5, 5, 5, 7, 11], 'exception.': [5, 5], 'diversity': [5, 7], 'bdfl.': [5, 9], 'return': [5, 7, 7, 11], 'exception': [5, 5, 7], 'self': [5, 7], 'dunder.': [5, 11], 'mercurial': [5, 7, 7, 7], 'bdfl': [5, 5, 11], 'generator.': [5], 'decorator': [5, 9, 9, 11], 'import.': [5, 7], 'future': [5, 11], 'klass': [5], 'community

### Discussion

`enumerate()` is a nice shortcut for situations where you might be inclined to keep your own counter variable: 

However, it's usually much more elegant (and less error-prone) to use `enumerate()` instead:

The value returned by `enumerate()` is an [instance of an enumerate object](https://docs.python.org/3/library/functions.html#enumerate), which is an iterator that returns successive tuples consisting of a counter and the value returned by calling `next()` on the sequence you’ve passed in.  
Although a minor point, it’s worth mentioning that sometimes it is easy to get tripped up when applying `enumerate()` to a sequence of tuples that are also being unpacked.  
To do it, you have to write code like this:

In [81]:
data = [ (1, 2), (3, 4), (5, 6), (7, 8) ]
# No errors this way:
for n, (x, y) in enumerate(data):
    # ...
    pass
# Error!:
for n, x, y in enumerate(data):
    # ... 
    pass

ValueError: not enough values to unpack (expected 3, got 2)

See, I told you in the comments in the code, sucka!

## [Iterating Over Multiple Sequences Simultaneously](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_iterating_over_multiple_sequences_simultaneously)

### Problem

You want to iterate over the items contained in more than one sequence at a time.

### Solution

To iterate over more than one sequence simultaneously, use the `zip()` function.

In [ ]:
xaxis = [1, 5, 4, 2, 10, 7]
yaxis = [101, 78, 37, 15, 62, 99]

for x, y in zip(xaxis, yaxis):
    print(x, y)

`zip(a, b)` works by creating an iterator that produces tuples `(x, y)` where x is taken from a and y is taken from b.  
Iteration stops whenever one of the input sequences is exhausted.  
Thus, the length of the iteration is the same as the length of the shortest input.

In [ ]:
a = [1, 2, 3]
b = ['w', 'x', 'y', 'z']
for i in zip(a, b):
    print(i)

 If the lengths of the sequences are different, you can use `itertools.zip_longest()`:

In [ ]:
from itertools import zip_longest

for i in zip_longest(a, b):
    print(i)

In [ ]:
for i in zip_longest(a, b, fillvalue=0):
    print(i)

### Discussion

`zip()` is commonly used whenever you need to pair data together.  
for example, suppose you have a list of column headers and column values like this:

In [ ]:
headers = ['name', 'shares', 'price']
values = ['ACME', 100, 490.1]

Using `zip()`, you can pair the values together to make a dictionary like this:

In [ ]:
s = dict(zip(headers, values))
print(s)

You can display the output differently, if you wish: 

In [ ]:
for name, val in zip(headers, values):
    print(name, '=', val)

`zip()` can also be passed more than two sequences as input.  
For this case, the resulting tuples have the same number of items in them as the number of input sequences.

In [ ]:
a = [1, 2, 3]
b = [10, 11, 12]
c = ['x', 'y', 'z']

for i in zip(a, b, c):
    print(i)

The result returned by zip is an iterator.  
Remember to use the `list()` function to store the paired values.

In [ ]:
zip(a, b)

In [ ]:
list(zip(a, b))

## [Iterating on Items in Separate Containers](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_iterating_on_items_in_separate_containers)

### Problem

You need to perform the same operation on many objects, but the objects are stored in different containers, and you’d like to avoid nested loops without losing the readability of your code.

### Solution

The `itertools.chain()` method can be used to simplify this task.  
It takes a list of iterables as input, and returns an iterator that effectively masks the fact that you’re really acting on multiple containers.  
To illustrate, consider this example:

In [ ]:
from itertools import chain

a = [1, 2, 3, 4]
b = ['x', 'y', 'z']
print([x for x in chain(a,b)])

In [ ]:
for x in chain(a,b):
    print(x)

A common use of `chain()` is in programs where you would like to perform certain operations on all of the items at once, but the items are pooled into different working sets.  
The following approach is more elegant than using two separate loops.

In [ ]:
active_items = set([1, 2, 2, 3])
inactive_items = set(['a', 'a', 'b', 'c'])
for thing in chain(active_items, inactive_items):
    print(thing)

### Discussion

`itertools.chain()` accepts one or more iterables as arguments.  
It then works by creating an iterator that successively consumes and returns the items produced by each of the supplied iterables you provided.  
It’s a subtle distinction, but `chain()` is more efficient than first combining the sequences and iterating.

In the first case, the operation `a + b` creates an entirely new sequence and additionally requires a and b to be of the same type.  
`chain()` performs no such operation, so it’s far more efficient with memory if the input sequences are large and it can be easily applied when the iterables in question are of different types.

## [Creating Data Processing Pipelines](http://chimera.labs.oreilly.com/books/1230000000393/ch04.html#_discussion_68)